# **DIGITAL DIARY**

Digital Diary is a small project that creates a separate diary for each user that is logged in to the system.

The application allows user authentication enabling users to signup, login and manage their diary entries.

The data is stored in .txt files to ensure persistent data sccrosss sessions

In [4]:
import os  # for files manipulation
from datetime import datetime   # to record date when data was entered

### **DIARY ENTRY CLASS**
- The diary entry will represent a single entry in the diary.
- An object will be created when ever a new entry is being entered or when we are loading the content from the text file to the entry list

In [5]:
class DiaryEntry:
    def __init__(self, title, content):
        self.title = title
        self.content = content
        self.timestamp = datetime.now()

    def update_content(self, content):
        self.content = content
        self.timestamp = datetime.now()

    def __str__(self):
        return f"{self.timestamp.strftime('%Y-%m-%d %H:%M:%S')}\nTitle: {self.title}\nContent: {self.content}\n"


### **DIARY CLASS**
- First I have created the diary's class that manages user's diary entries.
- The class will also allow saving, addition, updation, deletion of entries in a praticular diary.
- Each user's diary entries are stored in text files after their usernames.

**CODE DEMONSTRATION**
Some of the important methods of this class are described below for a better understanding:

#### Constructor
- takes 'username' as parameter.
- load existing entries from the text file for the given surname

#### Load Entries
- the method will load the existing content from the text file named "username_diary.txt"
- First the data is read line by line as each line represent one entry
- After getting the line data is parsed with the help of comma and entitites are separated by commas.
- Creates DiaryEntry objects and appends them to the entries list.
- Returns the list of entries.
- The method ensures that diary entries are loaded into memory when the Diary object is initialized so that user can interact with existing entries.

#### Save Entries
- Save the current list of diary entries to a text file named username_diary.txt
- Ensures that any changes to the diary entries (additions, updates, deletions) are persisted to the text file.

#### Add Entry
- Add a new DiaryEntry Object to the entries list.
- After that save entries method is called to update the list i nthe text file.  

In [16]:
class Diary:
    def __init__(self, username):
        self.username = username
        self.entries = self.load_entries()

    def load_entries(self):
        filename = f'{self.username}_diary.txt'
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                entries_data = f.read().strip().split('\n\n')
                entries = []
                for entry_data in entries_data:
                    if entry_data:
                        lines = entry_data.split('\n')
                        timestamp = lines[0]
                        title = lines[1].split(': ')[1]
                        content = '\n'.join(lines[2:])
                        entry = DiaryEntry(title, content)
                        entry.timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                        entries.append(entry)
                return entries
        return []

    def save_entries(self):
        filename = f'{self.username}_diary.txt'
        with open(filename, 'w') as f:
            for entry in self.entries:
                f.write(f"{entry.timestamp.strftime('%Y-%m-%d %H:%M:%S')}\nTitle: {entry.title}\n{entry.content}\n\n")

    def add_entry(self, entry):
        self.entries.append(entry)
        self.save_entries()

    def update_entry(self, title, new_content):
        for entry in self.entries:
            if entry.title == title:
                entry.update_content(new_content)
                self.save_entries()
                return True
        return False

    def delete_entry(self, title):
        self.entries = [entry for entry in self.entries if entry.title != title]
        self.save_entries()  # Save the updated entries to the file

    def list_entries(self):
        if not self.entries:
            print("No diary entries found.")
        for entry in self.entries:
            print(entry)

    def find_entry(self, title):
        for entry in self.entries:
            if entry.title == title:
                return entry
        return None


### **USER CLASS**
- In user class simply the constructor is saving the diary information of the user by creating a Diary Object and passing the username as an attribue.
- For user authentication I have created the verify password method that checks if user entered password is equal to the password present in the file.

In [17]:
class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.diary = Diary(username)

    def verify_password(self, password):
        return self.password == password

### **DIARY SYSTEM CLASS**
- The class manages the user registration, authentication and diary operations using the OOP concepts.
- Upon **initialization**, it loads user data from users.txt if the file exists, creating a dictionary where usernames map to User objects.
- The **signup method** allows new users to register by providing a username and password. It checks if the username already exists in self.users. If not, it creates a new User object and adds it to self.users, then saves the updated user data to users.txt.
- The **main_menu method** presents options for signing up, logging in, or exiting the program. Depending on the user's choice, it calls signup, login, or terminates the loop.
- After successful login, the **user_menu method** provides a menu for diary management specific to the logged-in user.
- Diary entries are managed through the Diary class, where methods like add_entry, update_entry, delete_entry, list_entries, and find_entry interact with entries stored in memory and persist them to username_diary.txt files.

In [18]:
class DiarySystem:
    def __init__(self):
        self.users = self.load_users()

    def load_users(self):
        if os.path.exists('users.txt'):
            with open('users.txt', 'r') as f:
                users_data = [line.strip().split(',') for line in f.readlines()]
                return {user[0]: User(user[0], user[1]) for user in users_data}
        return {}

    def save_users(self):
        with open('users.txt', 'w') as f:
            for user in self.users.values():
                f.write(f"{user.username},{user.password}\n")

    def signup(self, username, password):
        if username in self.users:
            print("Username alrdgeady exists. Please choose a different username.")
        else:
            self.users[username] = User(username, password)
            self.save_users()
            print("User registered successfully.")

    def login(self, username, password):
        user = self.users.get(username)
        if user and user.verify_password(password):
            print("Login successful.")
            return user
        else:
            print("Invalid username or password.")
            return None

    def main_menu(self):
        while True:
            print("\nMain Menu:")
            print("1. Sign Up")
            print("2. Log In")
            print("3. Exit")
            choice = input("Enter your choice: ")
            if choice == '1':
                username = input("Enter username: ")
                password = input("Enter password: ")
                self.signup(username, password)
            elif choice == '2':
                username = input("Enter username: ")
                password = input("Enter password: ")
                user = self.login(username, password)
                if user:
                    self.user_menu(user)
            elif choice == '3':
                print('Good Bye!')
                break
            else:
                print("Invalid choice. Please try again.")

    def user_menu(self, user):
        while True:
            print(f"\nDiary Menu for {user.username}:")
            print("1. Add Entry")
            print("2. Update Entry")
            print("3. Delete Entry")
            print("4. List Entries")
            print("5. Log Out")
            choice = input("Enter your choice: ")
            if choice == '1':
                title = input("Enter entry title: ")
                content = input("Enter entry content: ")
                entry = DiaryEntry(title, content)
                user.diary.add_entry(entry)
                print("Entry added successfully.")
            elif choice == '2':
                title = input("Enter entry title to update: ")
                new_content = input("Enter new content: ")
                if user.diary.update_entry(title, new_content):
                    print("Entry updated successfully.")
                else:
                    print("Entry not found.")
            elif choice == '3':
                title = input("Enter entry title to delete: ")
                user.diary.delete_entry(title)
                print("Entry deleted successfully.")
            elif choice == '4':
                user.diary.list_entries()
            elif choice == '5':
                break
            else:
                print("Invalid choice. Please try again.")


In [20]:
if __name__ == "__main__":
    diary_system = DiarySystem()
    diary_system.main_menu()


Main Menu:
1. Sign Up
2. Log In
3. Exit


Enter your choice:  1
Enter username:  Sara
Enter password:  gj768


User registered successfully.

Main Menu:
1. Sign Up
2. Log In
3. Exit


Enter your choice:  2
Enter username:  Sawa
Enter password:  sdf


Invalid username or password.

Main Menu:
1. Sign Up
2. Log In
3. Exit


...

2024-06-22 16:29:53
Title: My First Entry
Content: Today I am starting using my Digital Diary

2024-06-22 16:31:31
Title: My second Entry
Content: The second day


Diary Menu for Sara:
1. Add Entry
2. Update Entry
3. Delete Entry
4. List Entries
5. Log Out


Enter your choice:  5



Main Menu:
1. Sign Up
2. Log In
3. Exit


Enter your choice:  3


Good Bye!
